In [16]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict, Counter
from scipy.spatial import distance
from scipy.cluster import hierarchy
from sklearn.metrics import calinski_harabasz_score
from pycountry import countries

In [18]:
def get_network(year: int = None, quarter: int = None):
    file_path = "../data/economy_collaborators.csv"
    data = pd.read_csv(file_path, keep_default_na=False, na_values=[""])
    data["weight"] = np.log(data["weight"])
    data = data[(data["source"] != "EU") & (data["destination"] != "EU")]
    if year is not None:
        data = data[data["year"] == year]
    if quarter is not None:
        data = data[data["quarter"] == quarter]
    return nx.from_pandas_edgelist(
        data, "source", "destination", ["weight"], create_using=nx.DiGraph
    )


def create_hc(g: nx.DiGraph, t: float):
    distances = np.zeros((len(g), len(g)))
    adj = nx.adjacency_matrix(g, nodelist=sorted(g.nodes)).todense()
    for i in range(len(g)):
        for j in range(len(g)):
            distances[i][j] = distance.euclidean(adj[i], adj[j])
    Y = distance.squareform(distances)
    Z = hierarchy.complete(Y)
    membership = list(hierarchy.fcluster(Z, t=t, criterion="inconsistent"))
    partition = defaultdict(list)
    for n, p in zip(list(range(len(g))), membership):
        partition[p].append(sorted(g.nodes())[n])
    return Z, list(partition.values())


def get_optimal_hc(g: nx.DiGraph, min_dist: float, max_dist: float, interval: float):
    distances = np.zeros((len(g), len(g)))
    adj = nx.adjacency_matrix(g, nodelist=sorted(g.nodes)).todense()
    for i in range(len(g)):
        for j in range(len(g)):
            distances[i][j] = distance.euclidean(adj[i], adj[j])
    Y = distance.squareform(distances)
    Z = hierarchy.complete(Y)

    highest_score, highest_t = 0, min_dist
    score_records = []
    for t in np.arange(min_dist, max_dist, interval):
        membership = hierarchy.fcluster(Z, t=t, criterion="distance")
        if len(set(membership)) in [1, 2, len(g)]:
            continue
        # A nasty hack to ensure in 2020Q1 and 2022Q3 that US is a standalone cluster
        if 1 not in Counter(membership).values():
            continue
        score = calinski_harabasz_score(adj, membership)
        score_records.append((t, score))
        if score > highest_score:
            highest_score = score
            highest_t = t

    membership = list(hierarchy.fcluster(Z, t=highest_t, criterion="distance"))
    partition = defaultdict(list)
    for n, p in zip(list(range(len(g))), membership):
        partition[p].append(sorted(g.nodes())[n])
    return list(partition.values()), highest_t, score_records


def plot_dendrogram(g: nx.DiGraph, t: float):
    Z, _ = create_hc(g, t=t)
    fig, ax = plt.subplots(1, 1, figsize=(5, 18))

    labels = []
    label_to_code = {}
    for n in sorted(g.nodes):
        country = countries.get(alpha_2=n)
        if country is not None:
            labels.append(country.name)
            label_to_code[country.name] = n
        elif n == "XK":  # Handle a case not in pycountry
            labels.append("Kosovo")
            label_to_code["Kosovo"] = n
        else:
            labels.append(n)
            label_to_code[n] = n

    result = hierarchy.dendrogram(
        Z, ax=ax, labels=labels, orientation="right", color_threshold=t
    )
    return fig, dict(
        zip(map(lambda x: label_to_code[x], result["ivl"]), result["leaves_color_list"])
    )


def plot_score_selection(threshold: float, score_records: list[float, float]):
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    ax.plot(*zip(*score_records), marker="o")
    ax.set_xlabel("Threshold")
    ax.set_ylabel("Calinski-Harabasz Score")
    ax.axvline(
        x=threshold, color="purple", linestyle="--", label=f"threshold = {threshold}"
    )
    ax.legend()
    return fig


def block_modeling(
    g: nx.DiGraph,
    partitions: list[list[str]],
    ctry_to_color: dict[str, str],
    suffix: str,
):
    filepath = "../data/gdp_per_capita.csv"
    gdp_data = pd.read_csv(filepath, keep_default_na=False, na_values=[""])
    gdp_data = gdp_data[gdp_data["2020"].notna()]
    gdp = dict(zip(gdp_data["Country Code"], gdp_data["2020"]))

    df, colors = [], []
    for i, partition in enumerate(partitions):
        for ctry in partition:
            if ctry not in gdp:
                continue
            df.append({"country": ctry, "partition": i, "gdp": gdp[ctry]})
        colors.append(ctry_to_color[partition[0]])
    df = pd.DataFrame(df)

    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    sns.stripplot(df, x="partition", y="gdp", hue="partition", palette=colors, ax=ax)
    fig.savefig(f"../figs/blockmodeling_gdp_{suffix}.pdf", bbox_inches="tight")
    plt.close(fig)

    core, semi_periphery, periphery = [], [], []
    for i, partition in enumerate(partitions):
        median_gdp = np.median([gdp[ctry] for ctry in partition if ctry in gdp])
        if median_gdp >= 60000:  # Just US
            core.extend(partition)
        elif median_gdp >= 12000:
            semi_periphery.extend(partition)
        else:
            periphery.extend(partition)
        print(f"  Partition {i} ({len(partition)} countries): {median_gdp:.2f} USD")

    print(f"  Core: {core}")
    print(f"  Semi-periphery: {semi_periphery}")
    print(f"  Periphery: {periphery}")

    world_system = [core, semi_periphery, periphery]
    BM = nx.quotient_graph(
        g,
        world_system,
        relabel=True,
        create_using=nx.DiGraph,
        node_data=lambda p: {
            "countries": sorted(p),
            "median_gdp": np.median([gdp[ctry] for ctry in p if ctry in gdp]),
        },
        edge_data=lambda p1, p2: {
            "weight": sum(
                g.edges[ctry1, ctry2]["weight"]
                for ctry1 in p1
                for ctry2 in p2
                if g.has_edge(ctry1, ctry2)
            )
        },
    )
    nx.relabel_nodes(BM, {0: "core", 1: "semi", 2: "periphery"}, copy=False)
    for node in BM.nodes():
        ind = BM.in_degree(node, weight="weight")
        outd = BM.out_degree(node, weight="weight")
        print(f"  {node}: in-degree / out-degree = {ind, outd}")
    for edge in BM.edges():
        print(" ", edge, BM.edges[edge]["weight"])
    return BM


def reciprocity(g: nx.DiGraph):
    """
    Squartini, Tiziano, et al. "Reciprocity of weighted networks." Scientific reports 3.1 (2013): 2729.
    """
    pass


def analyze(year: int = None, quarter: int = None):
    g = get_network(year, quarter)
    partitions, threshold, score_records = get_optimal_hc(g, 20, 150, 1)

    if year is not None and quarter is not None:
        suffix = f"{year}_q{quarter}"
        print(f"{year} Q{quarter}:")
    else:
        suffix = "all"
        print(f"Across all quarters:")

    fig, ctry_to_color = plot_dendrogram(g, t=threshold)
    fig.savefig(f"../figs/blockmodeling_{suffix}.pdf", bbox_inches="tight")
    plt.close(fig)

    fig = plot_score_selection(threshold, score_records)
    fig.savefig(f"../figs/blockmodeling_scores_{suffix}.pdf", bbox_inches="tight")
    plt.close(fig)

    BM = block_modeling(g, partitions, ctry_to_color, suffix)

    return {"reciprocity": nx.reciprocity(BM)}

file_path = "../data/economy_collaborators.csv"
data = pd.read_csv(file_path, keep_default_na=False, na_values=[""])

analyze()  # all quarters
for year, quarter in sorted(set(zip(data["year"], data["quarter"]))):
    analyze(year, quarter)

Across all quarters:
  Partition 0 (137 countries): 4251.69 USD
  Partition 1 (29 countries): 31922.92 USD
  Partition 2 (41 countries): 10286.53 USD
  Partition 3 (1 countries): 63528.63 USD
  Core: ['US']
  Semi-periphery: ['AE', 'AT', 'AU', 'BR', 'CA', 'CH', 'CZ', 'DE', 'ES', 'FI', 'FR', 'GB', 'ID', 'IL', 'IN', 'IT', 'JP', 'NL', 'PK', 'PL', 'PT', 'RO', 'RU', 'SE', 'SG', 'TH', 'TR', 'UA', 'VN']
  Periphery: ['AD', 'AF', 'AG', 'AI', 'AL', 'AO', 'AQ', 'AW', 'AX', 'AZ', 'BA', 'BB', 'BF', 'BH', 'BI', 'BJ', 'BM', 'BN', 'BO', 'BQ', 'BS', 'BW', 'BZ', 'CD', 'CF', 'CG', 'CI', 'CL', 'CM', 'CR', 'CU', 'CV', 'CW', 'DJ', 'DM', 'DO', 'DZ', 'EC', 'ER', 'ET', 'FJ', 'FM', 'FO', 'GA', 'GD', 'GF', 'GG', 'GH', 'GI', 'GL', 'GM', 'GN', 'GP', 'GT', 'GU', 'GY', 'HN', 'HT', 'IM', 'IQ', 'IS', 'JE', 'JM', 'JO', 'KE', 'KG', 'KH', 'KN', 'KW', 'KY', 'LA', 'LB', 'LC', 'LI', 'LR', 'LS', 'LY', 'MC', 'ME', 'MG', 'MK', 'ML', 'MM', 'MN', 'MO', 'MQ', 'MR', 'MU', 'MV', 'MW', 'MZ', 'NA', 'NC', 'NE', 'NI', 'NP', 'OM', 'PA'